<a href="https://colab.research.google.com/github/ragingthunder511/da6401_assignment1/blob/main/Q2_3_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
pip install wandb

In [69]:
import numpy as np
import pandas as pd
import wandb
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.datasets import mnist
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [70]:
wandb.login()

True

In [71]:
class Start:
  def __init__(self):
    training_data,testing_data = fashion_mnist.load_data()
    self.train_x,self.train_y=training_data
    self.test_x,self.test_y=testing_data

    self.val_x = None
    self.val_y = None


  def data(self):
    return self.train_x,self.train_y,self.test_x,self.test_y

  def split_data(self):
    split_index = int(0.9 * self.train_x.shape[0])
    self.x_train_split, self.val_x =self. train_x[:split_index], self.train_x[split_index:]
    self.y_train_split, self.val_y =self. train_y[:split_index], self.train_y[split_index:]
    return self.x_train_split,self.val_x,self.y_train_split,self.val_y

  def modified_data(self):
    train_x=self.x_train_split.reshape(-1,28*28)/255.0
    test_x=self.test_x.reshape(-1,28*28)/255.0
    val_x=self.val_x.reshape(-1,28*28)/255.0
    train_y=np.eye(10)[self.y_train_split]
    test_y=np.eye(10)[self.test_y]
    val_y=np.eye(10)[self.val_y]
    return train_x,test_x,val_x,train_y,test_y,val_y

  def normalize_data(self):
    train_x=self.x_train_split.reshape(-1,28*28)/255.0
    test_x=self.test_x.reshape(-1,28*28)/255.0
    val_x=self.val_x.reshape(-1,28*28)/255.0
    return train_x,test_x,val_x

  # def show_data_samples(self):
  #   samples = {class_id: None for class_id in range(10)}
  #   count=0
  # # Class names for Fashion-MNIST
  #   classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
  # # Iterate through the training data to find one sample for each class
  #   for image, label in zip(self.train_x, self.train_y):
  #     if samples[label] is None:
  #       samples[label] = image
  #       count=count+1;
  #     if count==10: break

  #   wandb_images = []
  #   for i in range(10):
  #       wandb_images.append(wandb.Image(samples[i], caption=classes[i]))

  #   wandb.log({"Fashion-MNIST Samples": wandb_images})

  # # Plot the images in a grid
  #   plt.figure(figsize=(10, 5))
  #   for i in range(10):
  #     plt.subplot(2, 5, i + 1)
  #     plt.title(classes[i],fontsize=14,fontstyle='italic')
  #     plt.imshow(samples[i], cmap='gray')
  #     plt.axis('off')
  #   plt.suptitle('Fashion-MNIST Samples', fontsize=17,fontweight='bold')
  #   plt.show()

  def show_data_samples(self, sweep_name="sample_images_plot"):
    samples = {class_id: None for class_id in range(10)}
    count = 0

      # Class names for Fashion-MNIST
    classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

      # Collect one sample for each class
    for image, label in zip(self.train_x, self.train_y):
          if samples[label] is None:
              samples[label] = image
              count += 1
          if count == 10:
              break

      # Plot all samples in a single grid image
    plt.figure(figsize=(10, 5))
    for i, class_id in enumerate(samples.keys()):
          plt.subplot(2, 5, i + 1)
          plt.imshow(samples[class_id], cmap='gray')
          plt.title(classes[class_id], fontsize=12, fontstyle='italic')
          plt.axis('off')

    plt.suptitle(f'Fashion-MNIST Samples ({sweep_name})', fontsize=16, fontweight='bold')

      # Save the image and log it in WandB
    plt.savefig("fashion_mnist_samples.png")
    wandb.log({"Fashion-MNIST Samples": wandb.Image("fashion_mnist_samples.png")})
    plt.close()


Part 1 : CLASSES REQUIRED FOR MLFFNN ( Question 2 and Question 3 )

In [72]:
#Activation Functions
class Activation_Functions:
    def __init__(self, act):
        self.af = act.lower()

    #For Hidden Layers
    def activation_function(self, z):
        if self.af == 'relu':
            return np.maximum(0, z)
        elif self.af == 'tanh':
            z=np.clip(z,-500,500)
            return np.tanh(z)
        elif self.af == 'sigmoid':
            z=np.clip(z,-500,500)
            return 1 / (1 + np.exp(-z))
        elif self.af == 'identity':
            return z

    def activation_derivative(self, a):
        if self.af == 'relu':
            return (a > 0).astype(float)
        elif self.af == 'tanh':
            return 1 - a ** 2
        elif self.af == 'sigmoid':
            return a * (1 - a)
        elif self.af == 'identity':
            return np.ones_like(a)

    def ad2(self, z):
        if self.af == 'relu':
            return (z > 0).astype(int)
        elif self.af == 'tanh':
            ac = self.activation_function(z)
            return 1 - ac ** 2
        elif self.af == 'sigmoid':
            ac = self.activation_function(z)
            return ac * (1 - ac)
        elif self.af == 'identity':
            return np.ones_like(z)
    #For Output Layer
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)

#Intialization methods : To intialize weights
class Initializer_Methods:
  def __init__(self,m):
    self.mthd=m.lower()

  def initialize_weights(self, fin, fout, act):
       if self.mthd == 'xavier':
        if(act=='relu'):
          return np.random.randn(fin, fout) * np.sqrt(2 / fin)
        return np.random.randn(fin, fout) * np.sqrt(1 / fin)
       elif self.mthd == 'random':
        return np.random.randn(fin, fout) * 0.01

#Error functions and Accuracy calculation
class Eval_Metrics:
  def __init__(self):
      pass

  def cross_entropy_loss(self, y_true, y_pred):
      return -np.sum(y_true * np.log(y_pred + 1e-9)) / y_true.shape[0]

  def squared_error(self,y_true,y_pred):
      return np.mean(np.sum(np.square(y_true - y_pred), axis=1))

  def accuracy(self, y_true, y_pred):
      return np.mean(np.argmax(y_true, axis=1) == np.argmax(y_pred, axis=1))


# QUESTION 2: IMPLEMENTATION OF FEED FORWARD

In [73]:
class FF:
  def __init__(self, input_size, hidden_layers, output_size, activation='tanh', weight_init='random' , error_type='cross'):
    self.layers = [input_size] + hidden_layers + [output_size]
    self.activation = activation.lower()
    self.error=error_type

    ac=Activation_Functions(self.activation)
    self.af=ac.activation_function
    self.softmax=ac.softmax

    w_init=Initializer_Methods(weight_init)
    self.initialize_weights = w_init.initialize_weights

    eval=Eval_Metrics()
    self.cross_entropy_loss=eval.cross_entropy_loss

    self.weights = [self.initialize_weights(self.layers[i], self.layers[i + 1], self.activation) for i in range(len(self.layers) - 1)]
    self.biases = [np.zeros((1, self.layers[i + 1])) for i in range(len(self.layers) - 1)]

  def forward(self, x):
        x = x.reshape(x.shape[0], -1)
        a = x
        self.activations = [a]
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            a = self.af(np.dot(a, w) + b)
            self.activations.append(a)
        output = np.dot(a, self.weights[-1]) + self.biases[-1]
        self.activations.append(output)
        return self.softmax(output)


## Implementation of question 2 for 10 training data samples


In [74]:
# Example usage:
hidden_size = 3
hidden = [128]*hidden_size
output_size=10
input_size = 784
nn = FF(input_size,hidden,output_size)

x=Start()
trainx,testx,trainy,testy=x.data()
trainx,valx,trainy,valy=x.split_data()
x_train,x_test,x_val,y_train,y_test,y_val = x.modified_data()

# Generate a dummy Fashion-MNIST image (flattened)
x_sample = np.random.rand(1, 28*28)  # A single image
for i in range(10):
  output_probs = nn.forward(x_train[i][np.newaxis, :])
  print("For ",i+1,"th training sample , the output probabilities are:", output_probs)
  print("For verification: sum of probabilities = ",np.sum(output_probs))
  print("")

For  1 th training sample , the output probabilities are: [[0.10000906 0.0999721  0.10004404 0.09998662 0.09998444 0.10002164
  0.09998907 0.10000904 0.09997316 0.10001083]]
For verification: sum of probabilities =  1.0000000000000002

For  2 th training sample , the output probabilities are: [[0.09999711 0.1000084  0.10003889 0.09999799 0.099994   0.10002859
  0.09996909 0.09998656 0.09996441 0.10001496]]
For verification: sum of probabilities =  1.0

For  3 th training sample , the output probabilities are: [[0.10000802 0.10000936 0.10001058 0.10000148 0.10001257 0.10001268
  0.09998371 0.0999868  0.09997144 0.10000335]]
For verification: sum of probabilities =  1.0

For  4 th training sample , the output probabilities are: [[0.10000799 0.10000812 0.1000094  0.09999928 0.1000048  0.10002651
  0.09997719 0.09998164 0.09997414 0.10001094]]
For verification: sum of probabilities =  1.0

For  5 th training sample , the output probabilities are: [[0.10000743 0.10002159 0.10003976 0.100002

# QUESTION 3: IMPLEMENTATION OF FFNN MODEL

In [75]:
class FeedForwardNN:
    #Initialization of class data
    def __init__(self, input_size, hidden_layers, output_size, learning_rate=0.001, activation='sigmoid', weight_init='random', weight_decay=0.0 , error_type='cross'):
        """
        Initialize the neural network with given layer sizes and configurations.
        :param input_size: Number of input neurons (e.g., 28x28 for Fashion-MNIST)
        :param hidden_layers: List containing the number of neurons in each hidden layer
        :param output_size: Number of output neurons (10 for Fashion-MNIST classes)
        :param learning_rate: Learning rate for gradient descent
        :param activation: Activation function for hidden layers ('relu', 'tanh', 'sigmoid')
        :param weight_init: Weight initialization method ('random', 'xavier')
        :param weight_decay: L2 regularization parameter
        """
        self.layers = [input_size] + hidden_layers + [output_size]
        self.activation = activation.lower()
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.error=error_type
        self.weight_init=weight_init
        self.t = 0
        self.activations = []
        self.d_weights = []
        self.d_biases = []


        ac=Activation_Functions(self.activation)
        self.af=ac.activation_function
        self.ad1=ac.activation_derivative
        self.ad2=ac.ad2
        self.softmax=ac.softmax

        w_init=Initializer_Methods(self.weight_init)
        self.initialize_weights = w_init.initialize_weights

        eval=Eval_Metrics()
        self.cross_entropy_loss=eval.cross_entropy_loss
        self.squared_error=eval.squared_error
        self.accuracy=eval.accuracy


        # Initialize weights and biases
        self.weights = [self.initialize_weights(self.layers[i], self.layers[i + 1], weight_init) for i in range(len(self.layers) - 1)]
        self.biases = [np.zeros((1, self.layers[i + 1])) for i in range(len(self.layers) - 1)]

        # For the optimizers, initialize the necessary variables
        self.momentums = [np.zeros_like(w) for w in self.weights]
        self.biases_momentums = [np.zeros_like(b) for b in self.biases]
        self.velocity = [np.zeros_like(w) for w in self.weights]
        self.squared_gradients = [np.zeros_like(w) for w in self.weights]
        self.squared_biases = [np.zeros_like(w) for w in self.biases]
        self.m = [np.zeros_like(w) for w in self.weights]
        self.v = [np.zeros_like(w) for w in self.weights]
        self.biases_m = [np.zeros_like(w) for w in self.biases]
        self.biases_v = [np.zeros_like(w) for w in self.biases]


    def forward(self, x):
        x = x.reshape(x.shape[0], -1)
        a = x
        self.activations = [a]
        for w, b in zip(self.weights[:-1], self.biases[:-1]):
            a = self.af(np.dot(a, w) + b)
            self.activations.append(a)
        output = np.dot(a, self.weights[-1]) + self.biases[-1]
        self.activations.append(output)
        return self.softmax(output)

    def backward(self, x, y):
        m = x.shape[0]
        if(self.error=='cross'):
          d_output = self.activations[-1] - y
        elif(self.error=='squared'):
          d_output =( (self.activations[-1] - y)*2 ) / m

        d_w = np.dot(self.activations[-2].T, d_output) / m + self.weight_decay * self.weights[-1]
        d_b = np.sum(d_output, axis=0, keepdims=True) / m

        self.d_weights = [d_w]
        self.d_biases = [d_b]

        for i in range(len(self.layers) - 3, -1, -1):
            #d_activation = np.dot(d_output, self.weights[i + 1].T) * self.activation_derivative(np.dot(self.activations[i], self.weights[i]) + self.biases[i])
            d_activation = np.dot(d_output, self.weights[i + 1].T) * self.ad2(self.activations[i + 1])
            d_output = d_activation
            d_w = np.dot(self.activations[i].T, d_output) / m + self.weight_decay * self.weights[i]
            d_b = np.sum(d_output, axis=0, keepdims=True) / m
            self.d_weights.insert(0, d_w)
            self.d_biases.insert(0, d_b)

    def update_weights(self, optimizer="sgd"):
        """Update weights and biases using the specified optimization algorithm."""
        if optimizer == "sgd":
            self.sgd_optimizer()
        elif optimizer == "momentum":
            self.momentum_optimizer()
        elif optimizer == "nesterov":
            self.nesterov_optimizer()
        elif optimizer == "rmsprop":
            self.rmsprop_optimizer()
        elif optimizer == "adam":
            self.adam_optimizer()
        elif optimizer == "nadam":
            self.nadam_optimizer()

    #Optimizers
    def sgd_optimizer(self):
        """SGD update rule."""
        for i in range(len(self.weights)):
            self.weights[i] -= self.learning_rate * (self.d_weights[i]+self.weight_decay*self.weights[i])
            self.biases[i] -= self.learning_rate * self.d_biases[i]

   # def momentum_optimizer(self, beta=0.9):
   #    """Momentum-based gradient descent update."""
   #    for i in range(len(self.weights)):
   #        self.momentums[i] = beta * self.momentums[i] + (1 - beta) * self.d_weights[i]
   #        self.weights[i] -= self.learning_rate * self.momentums[i]
   #        self.biases[i] -= self.learning_rate * self.d_biases[i]


    #def momentum_optimizer(self, beta=0.9):
    #  """Momentum-based gradient descent update."""
    #  for i in range(len(self.weights)):
    #      self.momentums[i] = beta * self.momentums[i] - self.learning_rate * self.d_weights[i]
    #      self.weights[i] += self.momentums[i]
    #      self.biases_momentums[i] = beta * self.biases_momentums[i] - self.learning_rate * self.d_biases[i]
    #      self.biases[i] += self.biases_momentums[i]

    def momentum_optimizer(self, beta=0.9):
      """Momentum-based gradient descent update."""
      for i in range(len(self.weights)):
          self.momentums[i] = beta * self.momentums[i] + self.learning_rate * (self.d_weights[i]+self.weight_decay*self.weights[i])
          self.weights[i] -= self.learning_rate * self.momentums[i]
          self.biases_momentums[i] = beta * self.biases_momentums[i] + self.learning_rate * self.d_biases[i]
          self.biases[i] -= self.learning_rate * self.biases_momentums[i]


    def nesterov_optimizer(self, beta=0.9):
      """Nesterov Accelerated Gradient Descent (NAG)."""
      for i in range(len(self.weights)):
          lookahead_weights = self.weights[i] - beta * self.momentums[i]
          grad_w = self.d_weights[i]
          self.momentums[i] = beta * self.momentums[i] + self.learning_rate * (grad_w + self.weight_decay*lookahead_weights)
          self.weights[i] = lookahead_weights - self.learning_rate * self.momentums[i]

          lookahead_biases = self.biases[i] - beta * self.biases_momentums[i]
          grad_b = self.d_biases[i]
          self.biases_momentums[i] = beta * self.biases_momentums[i] + self.learning_rate * grad_b
          self.biases[i] = lookahead_biases - self.learning_rate * self.biases_momentums[i]

        # self.momentums[i] = ( beta * self.momentums[i] ) - ( self.learning_rate * self.d_weights[i] )
        # self.weights[i] += ( beta * self.momentums[i] ) - ( self.learning_rate * self.momentums[i] )
        # self.biases_momentums[i] = ( beta * self.biases_momentums[i] ) - ( self.learning_rate * self.d_biases[i] )
        # self.biases[i] += ( beta * self.biases_momentums[i] ) - ( self.learning_rate * self.biases_momentums[i] )


    def rmsprop_optimizer(self, beta=0.9, epsilon=1e-6):
      """RMSprop optimizer (fixed bias update issue)."""
      for i in range(len(self.weights)):
          self.squared_gradients[i] = beta * self.squared_gradients[i] + (1 - beta) * (self.d_weights[i] ** 2)
          self.squared_biases[i] = beta * self.squared_biases[i] + (1 - beta) * (self.d_biases[i] ** 2)
          self.weights[i] -= self.learning_rate * ((self.d_weights[i]+ (self.weight_decay * self.weights[i])) / (np.sqrt(self.squared_gradients[i]) + epsilon))
          self.biases[i] -= self.learning_rate * (self.d_biases[i] / (np.sqrt(self.squared_biases[i]) + epsilon))


    #def adam_optimizer(self, beta1=0.9, beta2=0.999, epsilon=1e-8):
    #    """Adam optimizer."""
    #    self.t += 1
    #    for i in range(len(self.weights)):
    #        self.m[i] = beta1 * self.m[i] + (1 - beta1) * self.d_weights[i]
    #        self.v[i] = beta2 * self.v[i] + (1 - beta2) * (self.d_weights[i] ** 2)
    #        m_hat = self.m[i] / (1 - (beta1 ** self.t))
    #        v_hat = self.v[i] / (1 - (beta2 ** self.t))
    #        self.weights[i] -= self.learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
    #        self.biases[i] -= self.learning_rate * self.d_biases[i]

    def adam_optimizer(self, beta1=0.9, beta2=0.999, epsilon=1e-6):
      """Adam optimizer."""
      self.t += 1
      for i in range(len(self.weights)):
          self.m[i] = beta1 * self.m[i] + (1 - beta1) * self.d_weights[i]
          self.v[i] = beta2 * self.v[i] + (1 - beta2) * (self.d_weights[i] ** 2)
          m_hat = self.m[i] / (1 - beta1 ** self.t)
          v_hat = self.v[i] / (1 - beta2 ** self.t)
          self.weights[i] -= self.learning_rate * (( m_hat /( (np.sqrt(v_hat) + epsilon)) + self.weight_decay * self.weights[i]) )

          self.biases_m[i] = beta1 * self.biases_m[i] + (1 - beta1) * self.d_biases[i]
          self.biases_v[i] = beta2 * self.biases_v[i] + (1 - beta2) * (self.d_biases[i] ** 2)
          biases_m_hat = self.biases_m[i] / (1 - beta1 ** self.t)
          biases_v_hat = self.biases_v[i] / (1 - beta2 ** self.t)
          self.biases[i] -= self.learning_rate * (( biases_m_hat / (np.sqrt(biases_v_hat) + epsilon))  )

    #def nadam_optimizer(self, beta1=0.9, beta2=0.999, epsilon=1e-8):
    #    """Nadam optimizer (combines Nesterov and Adam)."""
    #    self.t += 1
    #    for i in range(len(self.weights)):
    #        prev_momentum = self.momentums[i]
    #        self.momentums[i] = beta1 * self.momentums[i] + (1 - beta1) * self.d_weights[i]
    #        self.v[i] = beta2 * self.v[i] + (1 - beta2) * (self.d_weights[i] ** 2)
    #        m_hat = self.momentums[i] / (1 - (beta1 ** self.t))
    #        v_hat = self.v[i] / (1 - (beta2 ** self.t))
    #        self.weights[i] -= self.learning_rate * (m_hat + (1 - beta1) * prev_momentum) / (np.sqrt(v_hat) + epsilon)
    #        self.biases[i] -= self.learning_rate * self.d_biases[i]

    #def nadam_optimizer(self, beta1=0.9, beta2=0.999, epsilon=1e-8):
    #  """Nadam optimizer (combining Nesterov and Adam)."""
    #  self.t += 1
    #  for i in range(len(self.weights)):
    #      self.m[i] = beta1 * self.m[i] + (1 - beta1) * self.d_weights[i]
    #      self.v[i] = beta2 * self.v[i] + (1 - beta2) * (self.d_weights[i] ** 2)
    #      m_hat = self.m[i] / (1 - (beta1 ** self.t))
    #      v_hat = self.v[i] / (1 - (beta2 ** self.t))
    #      nesterov_m = beta1 * m_hat + (1 - beta1) * self.d_weights[i]
    #      self.weights[i] -= self.learning_rate * nesterov_m / (np.sqrt(v_hat) + epsilon)
    #      self.biases[i] -= self.learning_rate * self.d_biases[i]

    def nadam_optimizer(self, beta1=0.9, beta2=0.999, epsilon=1e-6):
      """Nadam optimizer (combining Nesterov and Adam)."""
      self.t += 1  # Time step for bias correction

      for i in range(len(self.weights)):
          self.m[i] = beta1 * self.m[i] + (1 - beta1) * self.d_weights[i]
          self.v[i] = beta2 * self.v[i] + (1 - beta2) * (self.d_weights[i] ** 2)
          m_hat = self.m[i] / (1 - (beta1 ** self.t))
          v_hat = self.v[i] / (1 - (beta2 ** self.t))
          nesterov_m = beta1 * m_hat + (1 - beta1) * self.d_weights[i]
          self.weights[i] -= self.learning_rate * (( nesterov_m / (np.sqrt(v_hat) + epsilon))+ self.weight_decay * self.weights[i] )

          self.biases_m[i] = beta1 * self.biases_m[i] + (1 - beta1) * self.d_biases[i]
          self.biases_v[i] = beta2 * self.biases_v[i] + (1 - beta2) * (self.d_biases[i] ** 2)
          biases_m_hat = self.biases_m[i] / (1 - (beta1 ** self.t))
          biases_v_hat = self.biases_v[i] / (1 - (beta2 ** self.t))
          nesterov_bias_m = beta1 * biases_m_hat + (1 - beta1) * self.d_biases[i]
          self.biases[i] -= self.learning_rate *( nesterov_bias_m / (np.sqrt(biases_v_hat) + epsilon) )


    # def train(self, x_train, y_train, epochs, batch_size, optimizer="sgd", validation_data=None):
    #     num_samples = x_train.shape[0]
    #     for epoch in range(epochs):
    #         total_loss = 0
    #         total_acc = 0
    #         permutation = np.random.permutation(num_samples)
    #         x_train = x_train[permutation]
    #         y_train = y_train[permutation]

    #         for i in range(0, num_samples, batch_size):
    #             x_batch = x_train[i:i+batch_size]
    #             y_batch = y_train[i:i+batch_size]
    #             output = self.forward(x_batch)
    #             if(self.error=='cross'):
    #                 loss = self.cross_entropy_loss(y_batch, output)
    #             elif(self.error=='squared'):
    #                 loss = self.squared_error(y_batch, output)
    #             acc = self.accuracy(y_batch, output)
    #             total_loss += loss
    #             total_acc += acc
    #             self.backward(x_batch, y_batch)
    #             self.update_weights(optimizer)

    #         val_loss = None
    #         val_acc = None
    #         if validation_data:
    #             x_val, y_val = validation_data
    #             val_output = self.forward(x_val)
    #             if(self.error=='cross'):
    #                 val_loss = self.cross_entropy_loss(y_val, val_output)
    #             elif(self.error=='squared'):
    #                 val_loss = self.squared_error(y_val, val_output)
    #             val_acc = self.accuracy(y_val, val_output)

    #         epoch_no = f"{epoch+1}/{epochs}"
    #         loss_no = total_loss / (num_samples // batch_size)
    #         acc_no = total_acc / (num_samples // batch_size)
    #         if validation_data:
    #             val_loss_no = val_loss
    #             val_acc_no = val_acc
    #         else:
    #             val_loss_no = None
    #             val_acc_no = None
    #         #print("Epoch:",epoch_no,"Loss:",loss_no,"Accuracy:",acc_no,"Val loss:",val_loss,"Val_accuracy:",val_acc)
    #         print(f"Epoch: {epoch+1}/{epochs}, Loss: {total_loss / (num_samples // batch_size)}, Accuracy: {(total_acc / (num_samples // batch_size))*100}, Val Loss: {val_loss}, Val Accuracy: {100*val_acc}")
    #         #wandb.log({"epoch": {epoch+1}/{epochs}, "loss": {total_loss / (num_samples // batch_size)}, "accuracy": {total_acc / (num_samples // batch_size)}, "val_loss": {val_loss}, "val_acc" : {val_acc}"})
    #         #wandb.log({"epoch": {epoch+1}/{epochs}, "loss": {total_loss / (num_samples // batch_size)}, "accuracy": {total_acc / (num_samples // batch_size)}})
    #         #wandb.log({"val_loss": {val_loss}, "val_acc" : {val_acc}})
    #         wandb.log({"epoch": epoch+1, "loss": loss_no, "accuracy": acc_no*100})
    #         wandb.log({"val_loss": val_loss_no, "val_acc" : val_acc*100})


    def train(self, x_train, y_train, epochs, batch_size, optimizer="sgd", validation_data=None):
      num_samples = x_train.shape[0]

      # Moving average for validation loss/accuracy
      val_loss_history = []
      val_acc_history = []

      for epoch in range(epochs):
          total_loss = 0
          total_acc = 0
          permutation = np.random.permutation(num_samples)
          x_train = x_train[permutation]
          y_train = y_train[permutation]

          # Mini-batch training loop
          for i in range(0, num_samples, batch_size):
              x_batch = x_train[i:i+batch_size]
              y_batch = y_train[i:i+batch_size]
              output = self.forward(x_batch)

              # Compute loss
              if self.error == 'cross':
                  loss = self.cross_entropy_loss(y_batch, output)
              elif self.error == 'squared':
                  loss = self.squared_error(y_batch, output)

              acc = self.accuracy(y_batch, output)
              total_loss += loss
              total_acc += acc

              # Backpropagation and weight update
              self.backward(x_batch, y_batch)
              self.update_weights(optimizer)

          # Compute epoch-level training loss & accuracy
          avg_train_loss = total_loss / (num_samples // batch_size)
          avg_train_acc = (total_acc / (num_samples // batch_size)) * 100

          # Validation computation
          val_loss_no, val_acc_no = None, None
          if validation_data:
              x_val, y_val = validation_data
              val_output = self.forward(x_val)

              # Compute validation loss
              if self.error == 'cross':
                  val_loss = self.cross_entropy_loss(y_val, val_output)
              elif self.error == 'squared':
                  val_loss = self.squared_error(y_val, val_output)

              val_acc = self.accuracy(y_val, val_output) * 100

              # Moving average for stability
              val_loss_history.append(val_loss)
              val_acc_history.append(val_acc)

              if len(val_loss_history) > 5:  # Maintain a window of last 5 epochs
                  val_loss_history.pop(0)
                  val_acc_history.pop(0)

              val_loss_no = sum(val_loss_history) / len(val_loss_history)
              val_acc_no = sum(val_acc_history) / len(val_acc_history)

          # Print results
          print(f"Epoch: {epoch+1}/{epochs}, Loss: {avg_train_loss:.4f}, Accuracy: {avg_train_acc:.2f}%, "
                f"Val Loss: {val_loss_no:.4f}, Val Accuracy: {val_acc_no:.2f}%")

          # Logging to WandB
          wandb.log({"epoch": epoch+1, "loss": avg_train_loss, "accuracy": avg_train_acc})
          if validation_data:
              wandb.log({"val_loss": val_loss_no, "val_acc": val_acc_no})



Question 3 ends above


CONFUSION MATRIX

In [76]:
def log_confusion_matrix(model, x_test, y_test, class_names):
    """Generate and log the confusion matrix in WandB."""

    # Get model predictions
    y_pred_probs = model.forward(x_test)  # Forward pass
    y_pred = np.argmax(y_pred_probs, axis=1)  # Convert to class labels
    y_true = np.argmax(y_test, axis=1)  # True class labels

    # Compute Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)

    # Plot Confusion Matrix
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt="d", cmap="coolwarm", xticklabels=class_names, yticklabels=class_names)
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    plt.title("Fashion-MNIST Confusion Matrix 🎨")

    # Saving the plot
    plt.savefig("confusion_matrix.png")

    #Logging confusion matrix in WandB
    wandb.log({"Confusion Matrix": wandb.Image("confusion_matrix.png")})

    # cm_table = wandb.Table(columns=["Actual", "Predicted"])
    # for actual, predicted in zip(y_true, y_pred):
    #     cm_table.add_data(class_names[actual], class_names[predicted])

    # wandb.log({"Confusion Matrix (Table)": cm_table})

    # Close the plot to prevent memory issues
    plt.close()

SWEEP CODE


In [ ]:
# WandB Sweep Configuration
sweep_configuration = {
    'method': "bayes",
    'metric': {'name': 'val_acc', 'goal': 'maximize'},
    'parameters': {
        'epochs': {'values': [5, 10]},
        'num_hidden_layers': {'values': [3, 4, 5]},
        'hidden_layer_size': {'values': [32, 64, 128]},
        'learning_rate': {'values': [1e-3, 1e-4]},
        'weight_decay': {'values': [0, 0.0005, 0.5]},
        #'optimizer_name': {'values': ['sgd','momentum','nesterov','rmsprop','adam','nadam']},
        'optimizer_name': {'values': ['sgd','momentum','nesterov','rmsprop','adam']},
        #'optimizer_name': {'values': ['momentum']},
        'batch_size': {'values': [16, 32, 64]},
        'init_type': {'values': ['random', 'xavier']},
        #'activation_type': {'values': ['sigmoid', 'tanh', 'relu', 'identity']},
        'activation_type': {'values': ['sigmoid', 'tanh', 'relu']},
        #'activation_type': {'values': ['tanh']},
        #'loss_type': {'values': ['cross', 'squared']}
        'loss_type': {'values': ['squared']}
    }
}

sweep_id = wandb.sweep(sweep_configuration, project="cs24m020_dla1_Q4")

Create sweep with ID: 8d1xh9o7
Sweep URL: https://wandb.ai/karekargrishma1234-iit-madras-/cs24m020_dla1_Q4/sweeps/8d1xh9o7


SWEEP CODE FOR BEST PERFORMING MODEL ( FOR CONFUSION MATRIX CALCULATION )

In [66]:
# WandB Best Sweep Configuration
sweep_best_configuration = {
    'method': "bayes",
    'metric': {'name': 'val_acc', 'goal': 'maximize'},
    'parameters': {
        'epochs': {'values': [10]},
        'num_hidden_layers': {'values': [3]},
        'hidden_layer_size': {'values': [128]},
        'learning_rate': {'values': [1e-4]},
        'weight_decay': {'values': [0]},
        'optimizer_name': {'values': ['rmsprop']},
        'batch_size': {'values': [32]},
        'init_type': {'values': [ 'xavier']},
        'activation_type': {'values': ['relu']},
        'loss_type': {'values': ['cross']}
    }
}

sweep_id = wandb.sweep(sweep_best_configuration, project="cs24m020_dla1_Q4")

Create sweep with ID: 13hw1axf
Sweep URL: https://wandb.ai/karekargrishma1234-iit-madras-/cs24m020_dla1_Q4/sweeps/13hw1axf


RUNNING SWEEPS

In [67]:
def train_sweep():
    # Initialize a new WandB run
    wandb.init(project="cs24m020_dla1",config=sweep_configuration)

    # Construct a meaningful run name
    run_name = f"h_{wandb.config.hidden_layer_size}_opt_{wandb.config.optimizer_name}_ac_{wandb.config.activation_type}_loss_{wandb.config.loss_type}_init_{wandb.config.init_type}_lr_{wandb.config.learning_rate}"

    # Set custom run name
    wandb.run.name = run_name
    wandb.run.save()  # Save the run name

    # Extract hyperparameters from the sweep config
    config = wandb.config
    num_hidden_layers = config.num_hidden_layers
    hidden_layer_size = config.hidden_layer_size
    hidden_layers = [hidden_layer_size] * num_hidden_layers  # Create hidden layer list
    learning_rate = config.learning_rate
    batch_size = config.batch_size
    optimizer_name = config.optimizer_name
    init_type = config.init_type
    activation_type = config.activation_type
    loss_type = config.loss_type
    epochs = config.epochs
    decay = config.weight_decay

    x=Start()
    trainx,testx,trainy,testy=x.data()
    trainx,valx,trainy,valy=x.split_data()
    x_train,x_test,x_val,y_train,y_test,y_val = x.modified_data()

    # Initialize the model
    model = FeedForwardNN(
        input_size=28*28, hidden_layers=hidden_layers, output_size=10, learning_rate=learning_rate, activation=activation_type, weight_init=init_type, weight_decay=decay,error_type=loss_type
    )

    # Train the model
    model.train(x_train, y_train, epochs, batch_size, optimizer=optimizer_name, validation_data=(x_val,y_val))

    classes = ["T-shirt", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

    x.show_data_samples()
    log_confusion_matrix(model, x_test, y_test, classes)

# Run the sweep
wandb.agent(sweep_id, train_sweep, count=1)

#self, input_size, hidden_layers, output_size, learning_rate=0.01, activation='relu', weight_init='random', weight_decay=0.0 , error_type='cross'
#self, x_train, y_train, epochs, batch_size, optimizer="sgd", validation_data=None

wandb: Agent Starting Run: y219typp with config:
wandb: 	activation_type: tanh
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 128
wandb: 	init_type: xavier
wandb: 	learning_rate: 0.0001
wandb: 	loss_type: cross
wandb: 	num_hidden_layers: 5
wandb: 	optimizer_name: adam
wandb: 	weight_decay: 0


Epoch: 1/10, Loss: 1.7884, Accuracy: 85.60%, Val Loss: 1.6563, Val Accuracy: 93.02%
Epoch: 2/10, Loss: 1.6385, Accuracy: 92.31%, Val Loss: 1.6189, Val Accuracy: 93.98%
Epoch: 3/10, Loss: 1.5965, Accuracy: 93.93%, Val Loss: 1.6008, Val Accuracy: 94.58%
Epoch: 4/10, Loss: 1.5759, Accuracy: 94.80%, Val Loss: 1.5884, Val Accuracy: 95.04%
Epoch: 5/10, Loss: 1.5624, Accuracy: 95.37%, Val Loss: 1.5781, Val Accuracy: 95.36%
Epoch: 6/10, Loss: 1.5524, Accuracy: 95.86%, Val Loss: 1.5549, Val Accuracy: 96.15%
Epoch: 7/10, Loss: 1.5447, Accuracy: 96.22%, Val Loss: 1.5453, Val Accuracy: 96.59%
Epoch: 8/10, Loss: 1.5391, Accuracy: 96.44%, Val Loss: 1.5375, Val Accuracy: 96.88%
Epoch: 9/10, Loss: 1.5339, Accuracy: 96.67%, Val Loss: 1.5322, Val Accuracy: 97.04%
Epoch: 10/10, Loss: 1.5299, Accuracy: 96.88%, Val Loss: 1.5285, Val Accuracy: 97.18%


<ipython-input-53-c4d35a2b2d33>:20: UserWarning: Glyph 127912 (\N{ARTIST PALETTE}) missing from font(s) DejaVu Sans.
  plt.savefig("confusion_matrix.png")


accuracy,▁▅▆▇▇▇████
epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▂▂▂▁▁▁▁
val_acc,▁▃▄▄▅▆▇███
val_loss,█▆▅▄▄▂▂▁▁▁
accuracy,96.875
epoch,10
loss,1.5299
val_acc,97.18
val_loss,1.52847
